In [28]:
import random as r
from math import degrees
from math import sqrt
from math import cos
from math import sin
from math import radians
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

vertical_length = 80
orizontal_length = 10
deg_var = 1
def create_data(sign1, sign2, n_cycles):
    def headingFromCoordinates(lat1, lon1, lat2, lon2): 
        if(lon2-lon1 < 0): #2nd and 3rd quadrant 
            heading =180 - degrees(np.arcsin((lat2-lat1)/(sqrt((lat2-lat1)**2 + (lon2-lon1)**2)))) 
        elif(lat2-lat1 < 0): #4th quadrant
            heading = 360 + degrees(np.arcsin((lat2-lat1)/(sqrt((lat2-lat1)**2 + (lon2-lon1)**2))))
        else:
            heading = degrees(np.arcsin((lat2-lat1)/(sqrt((lat2-lat1)**2 + (lon2-lon1)**2)))) #1st quadrant            
        return heading
    df = pd.read_csv('Data/Raw_boat_data')
    df = df[210:6830]
    
    spd = []
    dgr = []
    
    for i in range(n_cycles):
        for i in range(vertical_length):
            spd.append(np.random.normal(0.5, 0.3))
            dgr.append(np.random.normal(107.5, deg_var))
        for i in range(orizontal_length):
            spd.append(np.random.normal(0, 0.3))
            dgr.append(np.random.normal(7.5, deg_var))
        for i in range(vertical_length):
            spd.append(np.random.normal(-0.5, 0.3))
            dgr.append(np.random.normal(287.5, deg_var))
        for i in range(orizontal_length):
            spd.append(np.random.normal(0, 0.3))
            dgr.append(np.random.normal(7.5, deg_var))
    #         
    # for i in range(vertical_length):
    #     spd.append(np.random.normal(0.5, 0.3))
    #     dgr.append(np.random.normal(107.5, deg_var))
    # for i in range(orizontal_length):
    #     spd.append(np.random.normal(0, 0.3))
    #     dgr.append(np.random.normal(7.5, deg_var))
    # for i in range(vertical_length):
    #     spd.append(np.random.normal(-0.5, 0.3))
    #     dgr.append(np.random.normal(287.5, deg_var))
    # 

    m = np.mean(df['acceleration'])
    s = np.std(df['acceleration'])
    
    acc = np.random.normal(m, s-0.2, len(spd))
    
    m0c = np.random.normal(0, 0.5, len(spd))

    m1c = np.random.normal(0, 0.5, len(spd))

        
    lats = [45.3596665575, 45.36225589, 45.36226747, 45.35967972, 45.35969351, 45.36227073,
            45.36228557, 45.35970006, 45.35971575, 45.36230304, 45.36230583,
            45.35971167, 45.35972416, 45.36232427, 45.36232928, 45.35974926,
            45.359755  , 45.36234141, 45.3623531 , 45.35975891, 45.35976923,
            45.36236122, 45.36236823, 45.3597792384]
    
    lons = [11.020597651, 11.01979384, 11.01985461, 11.02066511, 11.02074007, 11.01991931,
            11.0199881 , 11.02079678, 11.02087983, 11.02006662, 11.02012215,
            11.02093044 , 11.02100831, 11.02020208, 11.02025402, 11.02108572,
            11.02115422, 11.02033527, 11.02038858, 11.02118961, 11.02124338,
            11.02045991, 11.02053796, 11.0213068174]
    
    lats = lats[:5*n_cycles]
    lons = lons[:5*n_cycles]
    
    variance_lat = 0.000005
    variance_lon = 0.000005
    
    def noise_lon(coord):
        noise = coord - np.random.normal(coord, variance_lon)
        return coord + noise
    
    def noise_lat(coord):
        noise = coord - np.random.normal(coord, variance_lat)
        return coord + noise
    
    k = np.linspace(-3,3, vertical_length+1) 
    
    def modify_angle(index, param):
        change = 5*param*2.5*np.exp(-np.power(k[index], 2.) / (2 * np.power(1, 2.)))
        return change
    
    for i, elem in enumerate(lats):
        lats[i] = noise_lat(elem)   
    
    for j,elem in enumerate(lons):
        lons[j] = noise_lon(elem)
    
    g_lat = []
    
    g_lon = []
    
    dgr_mod = []
    
    deviation_params = []
    deviation_index = 0
    for i in range(len(lats) - 1):
        start_lat = lats[i]
        start_lon = lons[i]
        g_lat.append(start_lat)
        g_lon.append(start_lon)
        l = sqrt((lons[i + 1] - lons[i]) ** 2 + (lats[i + 1] - lats[i]) ** 2)
        
        if i % 4 == 0:
            dl = l / vertical_length
            alpha = 180 - headingFromCoordinates(lats[i], lons[i], lats[i + 1], lons[i + 1])
            if sign1 == 1 or sign1 == 2:
                param = r.random()
                deviation_params.append(param)
            for j in range(1, vertical_length):
                if sign1 == 1:
                    alpha_m = alpha - modify_angle(j, deviation_params[deviation_index])
                elif sign1 == 2:
                    alpha_m = alpha + modify_angle(j, deviation_params[deviation_index])
                else:
                    alpha_m = alpha
                g_lon.append(noise_lon((start_lon - (j * dl * cos(radians(alpha_m))))))
                g_lat.append(noise_lat(start_lat + (j * dl * sin(radians(alpha_m)))))
            if sign1 == 1 or sign1 == 2:
                deviation_index += 1
        if i % 4 == 1:
            dl = l / orizontal_length
            alpha = headingFromCoordinates(lats[i], lons[i], lats[i + 1], lons[i + 1])
            for j in range(1, orizontal_length):
                g_lon.append(noise_lon(start_lon + (j * dl * cos(radians(alpha)))))
                g_lat.append(noise_lat(start_lat + (j * dl * sin(radians(alpha)))))
        if i % 4 == 2:
            dl = l / vertical_length
            alpha = headingFromCoordinates(lats[i], lons[i], lats[i + 1], lons[i + 1])
            if sign2 == 1 or sign2 == 2:
                param = r.random()
                deviation_params.append(param)
            for j in range(1, vertical_length):
                if sign2 == 1:
                    alpha_m = alpha - modify_angle(j, deviation_params[deviation_index])
                elif sign2 == 2:
                    alpha_m = alpha + modify_angle(j, deviation_params[deviation_index])
                else:
                    alpha_m = alpha
                g_lon.append(noise_lon(start_lon + (j * dl * cos(radians(alpha_m)))))
                g_lat.append(noise_lat(start_lat + (j * dl * sin(radians(alpha_m)))))
            if sign2 == 1 or sign2 == 2:
                deviation_index += 1
        if i % 4 == 3:
            dl = l / orizontal_length
            alpha = headingFromCoordinates(lats[i], lons[i], lats[i + 1], lons[i + 1])
            for j in range(1, orizontal_length):
                g_lon.append(noise_lon(start_lon + (j * dl * cos(radians(alpha)))))
                g_lat.append(noise_lat(start_lat + (j * dl * sin(radians(alpha)))))
    
    def modify_degrees():
        dgr_mod = []
        index = 1
        deviation_index = 0
        dev_bool = False
        for deg in dgr:
            if abs(deg - 107.5) < 10 and sign1 == 1:
                dgr_mod.append(deg - modify_angle(index,deviation_params[deviation_index]))
                index += 1
                dev_bool = True
            elif abs(deg - 107.5) < 10 and sign1 == 2:
                dgr_mod.append(deg + modify_angle(index,deviation_params[deviation_index]))
                index += 1
                dev_bool = True
            elif abs(deg - 287.5) < 10 and sign2 == 1:
                dgr_mod.append(deg - modify_angle(index,deviation_params[deviation_index]))
                index += 1
                dev_bool = True
            elif abs(deg - 287.5) < 10 and sign2 == 2:
                dgr_mod.append(deg - modify_angle(index,deviation_params[deviation_index]))
                index += 1
                dev_bool = True
            else:
                dgr_mod.append(deg)
                index = 1
                if dev_bool:    
                    deviation_index += 1
                    dev_bool = False
        return dgr_mod
    
    dgr_mod = modify_degrees()
    sins = []
    cosins = []
    for i, degree in enumerate(dgr_mod):
        sins.append(np.sin(np.deg2rad(degree)))
        cosins.append(np.cos(np.deg2rad(degree)))
           
    columns = ["Sin", "Cosin",  
               "Lat", "Lon"]
    
    
    #plt.plot(g_lon, g_lat)
    # plt.show()
    # plt.title(str(sign1)+
    #             str(sign2)+":"+str(choice))
    # plt.show()
    #  
    #plt.plot(dgr_mod)
    #plt.show()
    

    
    df = pd.DataFrame(columns=columns)
    #df["Speed"] = spd
    df["Sin"] = sins
    df["Cosin"] = cosins
    #df["Acceleration"] = acc
    #df["M0C"] = m0c
    #df["M1C"] = m1c
    df["Lat"] = g_lat[:len(spd)]
    df["Lon"] = g_lon[:len(spd)]
    
    #df = df.drop(df.index[-10:])
    
    return df


In [30]:
def save_in_csv(df_list):
    for index, run in enumerate(df_list):
        run.to_csv("Mix_sequences_var_length/run^"+str(index))
            
    

df = []
nom_list = []
for i in range(300):
    choice = np.random.choice([0, 1, 2, 3, 4, 5, 6, 7, 8], 
                              p=[0.6, 0.05, 0.05, 0.05, 
                                 0.05, 0.05, 0.05, 0.05,
                                 0.05])
    n_cycles = np.random.choice([1, 2, 3, 4, 5], 
                                p=[0.2, 0.2, 0.2, 0.2, 0.2])
    if choice == 1:
        nom_list.append(choice)
        data = create_data(sign1=1, sign2=0, n_cycles=n_cycles)
    if choice == 2:
        nom_list.append(choice)
        data = create_data(sign1=2, sign2=0, n_cycles=n_cycles)
    if choice == 3:
        nom_list.append(choice)
        data = create_data(sign1=0, sign2=1, n_cycles=n_cycles)
    if choice == 4:
        nom_list.append(choice)
        data = create_data(sign1=0, sign2=2, n_cycles=n_cycles)
    if choice == 5:
        nom_list.append(choice)
        data = create_data(sign1=1, sign2=1, n_cycles=n_cycles)
    if choice == 6:
        nom_list.append(choice)
        data = create_data(sign1=2, sign2=2, n_cycles=n_cycles)
    if choice == 7:
        nom_list.append(choice)
        data = create_data(sign1=2, sign2=1, n_cycles=n_cycles)
    if choice == 8:
        nom_list.append(choice)
        data = create_data(sign1=1, sign2=2, n_cycles=n_cycles)
    if choice == 0:
        nom_list.append(choice)
        data = create_data(sign1=0, sign2=0, n_cycles=n_cycles)
    df.append(data)

label_df = pd.DataFrame(np.array(nom_list), columns=['label'])
 
save_in_csv(df)

In [15]:
df = create_data(sign1=0, sign2=0, n_cycles=1)
nom_list = []
n_cycles = 1
df = df.iloc[0:0]
lab_unseen = []

for k in range(3):
    for i in range(5):
        lab_unseen.append(0)
        df = df.append(create_data(sign1=0, sign2=0, n_cycles=n_cycles))
    for i in range(5):
        lab_unseen.append(1)
        df = df.append(create_data(sign1=1, sign2=0, n_cycles=n_cycles))
    for i in range(5):
        lab_unseen.append(2)
        df = df.append(create_data(sign1=2, sign2=0, n_cycles=n_cycles))
    for i in range(5):
        lab_unseen.append(3)
        df = df.append(create_data(sign1=0, sign2=1, n_cycles=n_cycles))
    for i in range(5):
        lab_unseen.append(4)
        df = df.append(create_data(sign1=0, sign2=2, n_cycles=n_cycles))
    for i in range(5):
        lab_unseen.append(5)
        df = df.append(create_data(sign1=1, sign2=1, n_cycles=n_cycles))
    for i in range(5):
        lab_unseen.append(6)
        df = df.append(create_data(sign1=2, sign2=2, n_cycles=n_cycles))
    for i in range(5):
        lab_unseen.append(7)
        df = df.append(create_data(sign1=2, sign2=1, n_cycles=n_cycles))
    for i in range(5):
        lab_unseen.append(8)
        df = df.append(create_data(sign1=1, sign2=2, n_cycles=n_cycles))

label_df = pd.DataFrame(np.array(lab_unseen), columns=['label'])
save = True
if save:
    df.to_csv("Data/Boat_nominal_data/Boat_unseen.csv")
    label_df.to_csv("Data/Boat_nominal_data/Boat_unseen_labels.csv")


In [78]:

validation = True
# anomalous = True
if not validation:
    df.to_csv("Data/Boat_nominal_data/Boat_sequences_mix.csv")
    label_df.to_csv("Data/Boat_nominal_data/Boat_mix_labels.csv")
if validation:
    df.to_csv("Data/Boat_nominal_data/Boat_sequence_mix_val.csv")

# if anomalous:
#     df.to_csv("Data/Boat_anom_sequence.csv")
